In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()

In [3]:
Data_dir = "D:/recommender_systems/lastfm/lastfm-dataset-360K"

In [4]:
import os
import pyspark.sql.functions as F

In [5]:
plays = (
    spark
    .read
    .csv(
        os.path.join(Data_dir, "usersha1-artmbid-artname-plays.tsv"),
        header = True,
        inferSchema = True,
        sep = '\t'
    )
    .sample(withReplacement = False, fraction = 1.0, seed = 42)    
    .toDF("user", "artist", "artist_name", "plays")
    .cache()
    )

In [6]:
plays.show()


+--------------------+--------------------+--------------------+------+
|                user|              artist|         artist_name| plays|
+--------------------+--------------------+--------------------+------+
|00000c289a1829a80...|f2fb0ff0-5679-42e...|           die Ärzte|1099.0|
|00000c289a1829a80...|b3ae82c2-e60b-455...|   melissa etheridge| 897.0|
|00000c289a1829a80...|3d6bbeb7-f90e-4d1...|           elvenking| 717.0|
|00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706.0|
|00000c289a1829a80...|8bfac288-ccc5-448...|red hot chili pep...| 691.0|
|00000c289a1829a80...|6531c8b1-76ea-414...|              magica| 545.0|
|00000c289a1829a80...|21f3573f-10cf-44b...|the black dahlia ...| 507.0|
|00000c289a1829a80...|c5db90c4-580d-4f3...|         the murmurs| 424.0|
|00000c289a1829a80...|0639533a-0402-40b...|          lunachicks| 403.0|
|00000c289a1829a80...|a342964d-ca53-4e5...|    walls of jericho| 393.0|
|00000c289a1829a80...|f779ed95-66c8-449...|      letzte instanz|

In [14]:
total_users = plays.select("user").distinct().count()
total_artist = plays.select("artist_name").distinct().count()
total_samples = plays.count()
print("Всего пользователей: ", total_users)
print("Всего исполнителей: ", total_artist)
print("Всего пар пользователь-исполнитель: ", total_samples)
print("Процент ненулевых пар: ", total_users / total_users / total_artist)

Всего пользователей:  359349
Всего исполнителей:  292577
Всего пар пользователь-исполнитель:  17559529
Процент ненулевых пар:  3.4179036629673555e-06


In [8]:
print("Всего исполнителей: ", total_artist)

Всего исполнителей:  292577


In [9]:
print("Всего пар пользователь-исполнитель: ", total_samples)

Всего пар пользователь-исполнитель:  17559529


In [10]:
print("Процент ненулевых пар: ", total_users / total_users / total_artist)

Процент ненулевых пар:  3.4179036629673555e-06


In [11]:
plays.select("plays").describe().show()

+-------+-----------------+
|summary|            plays|
+-------+-----------------+
|  count|         17559517|
|   mean|215.1853181952556|
| stddev|614.4478103103975|
|    min|              0.0|
|    max|         419157.0|
+-------+-----------------+



In [12]:
plays.where("plays < 1").show(truncate = False)

+----------------------------------------+------+-----------+-----+
|user                                    |artist|artist_name|plays|
+----------------------------------------+------+-----------+-----+
|cbec2d2763c856034f5689ba13ca8addb9d24984|null  |28         |0.0  |
+----------------------------------------+------+-----------+-----+



In [15]:
plays.select(F.length("artist").alias("1")).distinct().orderBy("1").show()

+----+
|   1|
+----+
|null|
|   2|
|   3|
|  15|
|  35|
|  36|
|  37|
+----+



In [15]:
plays.where("LENGTH(artist) < 35").select("artist", "artist_name").distinct().show()

+---------------+--------------+
|         artist|   artist_name|
+---------------+--------------+
|            211| 000 promises.|
|             96|    000 things|
|            313|    000 things|
|rock / a30a400a|blind the fold|
+---------------+--------------+



In [16]:
plays.where("LENGTH(artist) > 35").select("artist", "artist_name").distinct().show()

ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение